In [174]:
import os
import sys
import numpy as np
sys.path.append('..')
import requests
import json
from pytube import YouTube
import youtube_dl
import cv2
from os import listdir
import shutil
import copy

In [185]:
with open('../data/activity_net.v1-2.min.json') as data_file:    
    data_json = json.load(data_file)

In [189]:
parent_classes = {}
parent_classes_name = {}
visited_by_node_id = {}
class_root_count = 0
for item in data_json['taxonomy']:
    if item['parentName'] == 'Root':
        class_root_count += 1
        item['idx'] = class_root_count
        parent_classes[item['nodeId']] = item
        parent_classes_name[item['nodeName']] = item
        visited_by_node_id[item['nodeId']] = 1

by_level = {0: parent_classes}
by_level_name = {0: parent_classes_name}
levels = 4
for level in range(1, levels + 1):
    class_level_count = 0
    for item in data_json['taxonomy']:
        if item['parentId'] in by_level[level - 1]:
            visited_by_node_id[item['nodeId']] = 1
            class_level_count += 1
            if level not in by_level:
                by_level[level] = {item['nodeId']: item}
                by_level_name[level] = {item['nodeName']: item}
            else:
                by_level[level][item['nodeId']] = item
                by_level_name[level][item['nodeName']] = item
                
            by_level[level][item['nodeId']]['idx'] = class_level_count
            by_level_name[level][item['nodeName']]['idx'] = class_level_count

for key, item in by_level.items():
    print('level: ', key, len(item))

level:  0 5
level:  1 12
level:  2 33
level:  3 100


In [190]:
path = '../data/'
classes_training = 0
count_labels = 0
videos_metadata = {}
examples_by_level = {0: 0, 1: 0, 2: 0, 3: 0}
label_found = {}
for key, item in data_json['database'].items():
    video_id = key
    for item in data_json['database'][video_id]['annotations']:
        count_labels += 1
        if item['label'] not in videos_metadata:
            classes_training += 1
            videos_metadata[item['label']] = {
                'examples': 1,
                'vid_secs': item['segment'][1] - item['segment'][0],
                'idx': classes_training
            }
        else:
            videos_metadata[item['label']]['examples'] += 1
            videos_metadata[item['label']]['vid_secs'] += item['segment'][1] - item['segment'][0]
        
        for level, nodes_level in by_level.items():
            found = False
            for node_keys, node_items in nodes_level.items():
                if item['label'] == node_items['nodeName']:
                    examples_by_level[level] += 1
                    label_found[item['label']] = 1
                    found = True
                    break
            if found:
                break


In [191]:
for key, item in videos_metadata.items():
    level = 3
    global_key = copy.deepcopy(key)
    while level > 0:
        current_parent = by_level_name[level - 1][by_level_name[level][key]['parentName']]
        videos_metadata[global_key]['level_' + str(level - 1)] = {
            'idx': current_parent['idx'],
            'name': current_parent['nodeName']
        }
        key = by_level_name[level][key]['parentName']
        level -= 1

videos_metadata['classes_amount'] = {}
for key, item in by_level.items():
    videos_metadata['classes_amount']['level_' + str(key)] = len(item)

videos_metadata

{'Archery': {'examples': 153,
  'idx': 7,
  'level_0': {'idx': 2, 'name': 'Sports, Exercise, and Recreation'},
  'level_1': {'idx': 2,
   'name': 'Participating in Sports, Exercise, or Recreation'},
  'level_2': {'idx': 6, 'name': 'Playing sports'},
  'vid_secs': 4640.250529999997},
 'Ballet': {'examples': 100,
  'idx': 8,
  'level_0': {'idx': 3, 'name': 'Socializing, Relaxing, and Leisure'},
  'level_1': {'idx': 3, 'name': 'Arts and Entertainment'},
  'level_2': {'idx': 31, 'name': 'Attending arts and entertainment'},
  'vid_secs': 6961.7527500000015},
 'Bathing dog': {'examples': 68,
  'idx': 29,
  'level_0': {'idx': 5, 'name': 'Household Activities'},
  'level_1': {'idx': 6, 'name': 'Animals and Pets'},
  'level_2': {'idx': 11,
   'name': 'Care for animals and pets (not veterinary care)'},
  'vid_secs': 3473.006159999999},
 'Belly dance': {'examples': 75,
  'idx': 73,
  'level_0': {'idx': 3, 'name': 'Socializing, Relaxing, and Leisure'},
  'level_1': {'idx': 3, 'name': 'Arts and Ent

In [192]:
with open(path + '/training_meta_data_reduced.json', 'w') as outfile:
    json.dump(videos_metadata, outfile)

In [193]:
videos_metadata

{'Archery': {'examples': 153,
  'idx': 7,
  'level_0': {'idx': 2, 'name': 'Sports, Exercise, and Recreation'},
  'level_1': {'idx': 2,
   'name': 'Participating in Sports, Exercise, or Recreation'},
  'level_2': {'idx': 6, 'name': 'Playing sports'},
  'vid_secs': 4640.250529999997},
 'Ballet': {'examples': 100,
  'idx': 8,
  'level_0': {'idx': 3, 'name': 'Socializing, Relaxing, and Leisure'},
  'level_1': {'idx': 3, 'name': 'Arts and Entertainment'},
  'level_2': {'idx': 31, 'name': 'Attending arts and entertainment'},
  'vid_secs': 6961.7527500000015},
 'Bathing dog': {'examples': 68,
  'idx': 29,
  'level_0': {'idx': 5, 'name': 'Household Activities'},
  'level_1': {'idx': 6, 'name': 'Animals and Pets'},
  'level_2': {'idx': 11,
   'name': 'Care for animals and pets (not veterinary care)'},
  'vid_secs': 3473.006159999999},
 'Belly dance': {'examples': 75,
  'idx': 73,
  'level_0': {'idx': 3, 'name': 'Socializing, Relaxing, and Leisure'},
  'level_1': {'idx': 3, 'name': 'Arts and Ent